In [ ]:
from pydantic import BaseModel, Field
from typing import List

class Slide(BaseModel):
    title: str = Field(
        description="title for this slide.",
    )
    text: str = Field(
        description="text for this slide.",
    )

class Slides(BaseModel):
    slides: List[Slide] = Field(
        description="Slides to present the document in a presentation.",
    )

In [ ]:
with open("texts/sample.txt", "r") as f:
    doc =f.read()

In [ ]:
from google import genai
from pydantic import BaseModel, Field
from typing import List, Optional

client = genai.Client()

prompt = """
make slides for the following text.
{text}
"""

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=prompt.format(text=doc),
    config={
        "response_mime_type": "application/json",
        "response_json_schema": Slides.model_json_schema(),
    },
)

res = Slides.model_validate_json(response.text)
print(res)

In [ ]:
len(res.slides)

In [ ]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()

OPENROUTER_API_KEY = os.environ["OPENROUTER_API_KEY"]


In [ ]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "presentation_slides",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "slides": {
                    "type": "array",
                    "description": "Slides to present the document in a presentation.",
                    "items": {
                        "type": "object",
                        "properties": {
                            "title": {
                                "type": "string",
                                "description": "title for this slide."
                            },
                            "text": {
                                "type": "string",
                                "description": "text for this slide."
                            }
                        },
                        "required": ["title", "text"],
                        "additionalProperties": False
                    }
                }
            },
            "required": ["slides"],
            "additionalProperties": False
        }
    }
}

In [ ]:
def generate_structured_output(prompt: str, model: str) -> dict:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "response_format": response_format
    }
    response = requests.post(url, headers=headers, json=payload)
    result = response.json()
    return result

In [ ]:
prompt = """
make slides for the following text.
{text}
"""

In [ ]:
res = generate_structured_output(prompt.format(text=doc), "google/gemini-3-pro-preview")

In [ ]:
slides = res["choices"][0]["message"]["content"]

In [ ]:
slides

In [ ]:
import json

In [ ]:
json_data =json.loads(slides)

In [ ]:
len(json_data['slides'])


In [ ]:
json_data['slides']